# ProWave - WaveNet-based Protein Generation

Authors: Hans Jakob Damsgaard & Lucas Balling

02456 Deep Learning project: ProGen

## Initialization

Run the commmand below if you have not yet installed the [TAPE project](https://github.com/songlab-cal/tape).

In [ ]:
#!pip install tape_proteins

#### Importing needed packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import tape

#### Importing the data

We were unable to make the data download script, `download_data.sh`, run from Jupyter, so instead we ran it manually and simply placed the resulting files in the right folder for TAPE to find them. We import all the data in the LMDB format as it is most easily worked with in Python.

In [7]:
from tape.datasets import LanguageModelingDataset

# Data stored under `<data-path>/data`
#data_path = '/Users/lucasballing/Desktop/DeepLearningProject/prowave-main/data/'
data_path = 'E:/Pfam/data/'
train_data   = LanguageModelingDataset(data_path, 'train')
valid_data   = LanguageModelingDataset(data_path, 'valid')
holdout_data = LanguageModelingDataset(data_path, 'holdout')

#### Understanding data features

To get a good understanding of the data provided in the imported dataset, we provide plots of certain features and their ranges. Data is already split into the three required subsets; train, validation, and holdout by TAPE, so it is also interesting to understand this split.

In [8]:
# Split sizes
print(f'Training data has shape ({len(train_data)}, {len(train_data[0])})')
print(f'Validation data has shape ({len(valid_data)}, {len(valid_data[0])})')
print(f'Holdout data has shape ({len(holdout_data)}, {len(holdout_data[0])})')

# Original data columns
from tape.datasets import LMDBDataset
lmdb_train = LMDBDataset(data_path+'pfam/pfam_train.lmdb')
print(f'File data entries look like this: {lmdb_train[0]}')
del lmdb_train

# Data columns - all subsets are taken from the same overall dataset, so the columns are the same
# From combining information from LMDBDataset and LanguageModelingDataset, we know the columns are
# - IUPAC-encoded protein string
# - Input mask (for masked-token prediction)
# - Protein clan
# - Protein family
# The protein ID (i.e., its number within its clan and family) is not included
print(f'Encoded data entries look like this: {train_data[0]}')

Training data has shape (32593668, 4)
Validation data has shape (1715454, 4)
Holdout data has shape (44311, 4)
File data entries look like this: {'primary': 'GCTVEDRCLIGMGAILLNGCVIGSGSLVAAGALITQ', 'protein_length': 36, 'clan': 433, 'family': 9122, 'id': '0'}
Encoded data entries look like this: (array([ 2, 11,  7, 23, 25,  9,  8, 21,  7, 15, 13, 11, 16, 11,  5, 13, 15,
       15, 17, 11,  7, 25, 13, 11, 22, 11, 22, 15, 25,  5,  5, 11,  5, 15,
       13, 23, 20,  3], dtype=int64), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), 433, 9122)


In [9]:
def setify(data):
    """
    Produces summary statistics for the provided datasets.

    Args:
     `data`: a list of datasets

    Returns a tuple of three lists;
     `uniques` representing sets of clans and families in each of the datasets
     `perclan` representing tuples of clan ID and protein count
     `perfam`  representing tuples of family ID and protein count
    """
    uniques = [[set() for _ in range(len(data))] for _ in range(2)]
    perclan = [{} for _ in range(len(data))]
    perfam  = [{} for _ in range(len(data))]
    for index, d in enumerate(data):
        for i in range(len(d)):
            # Fetch this entry
            row  = d[i]
            clan = row[2]
            fam  = row[3]

            # Add clan and family IDs to sets
            uniques[0][index].add(clan) # add clan
            uniques[1][index].add(fam) # add family

            # Count proteins in this clan
            if clan not in perclan[index]:
                perclan[index][clan] = 1
            else:
                perclan[index][clan] += 1

            # Count proteins in this family
            if fam not in perfam[index]:
                perfam[index][fam] = 1
            else:
                perfam[index][fam] += 1

    return uniques, [x.items() for x in perclan], [x.items() for x in perfam]


In [ ]:
# Fetch results from all splits
results = setify([train_data, valid_data, holdout_data])

# Clans in splits
clans = results[0][0]
print(f'Unique clans in training data {len(clans[0])}')
print(f'Unique clans in validation data {len(clans[1])}')
print(f'Unique clans in holdout data {len(clans[2])}')

# Families in splits
families = results[0][1]
print(f'Unique families in training data {len(families[0])}')
print(f'Unique families in validation data {len(families[1])}')
print(f'Unique families in holdout data {len(families[2])}')

# PRINTS:
# Unique clans in training data 623
# Unique clans in validation data 623
# Unique clans in holdout data 8
# Unique families in training data 17737
# Unique families in validation data 15974
# Unique families in holdout data 28


#### Creating some histograms
In this section, we plot some histograms of the datasets to visulise the distribution of clan and family IDs. 

In [ ]:
# Histograms of protein counts in clans
# TRAINING
df = pd.DataFrame(results[1][0], columns=['Clan', 'Count'])
sns.displot(df, x='Clan')
plt.title('Training - Protein count vs Clan')
plt.show()

# VALIDATION
df = pd.DataFrame(results[1][1], columns=['Clan', 'Count'])
sns.displot(df, x='Clan')
plt.title('Validation - Protein count vs Clan')
plt.show()

# HOLDOUT
df = pd.DataFrame(results[1][2], columns=['Clan', 'Count'])
sns.displot(df, x='Clan')
plt.title('Holdout - Protein count vs Clan')
plt.show()

# Histograms of protein counts in families
# TRAINING
df = pd.DataFrame(results[2][0], columns=['Family', 'Count'])
sns.displot(df, x='Family')
plt.title('Training - Protein count vs Clan')
plt.show()

# VALIDATION
df = pd.DataFrame(results[2][1], columns=['Family', 'Count'])
sns.displot(df, x='Family')
plt.title('Validation - Protein count vs Clan')
plt.show()

# HOLDOUT
df = pd.DataFrame(results[2][2], columns=['Family', 'Count'])
sns.displot(df, x='Family')
plt.title('Holdout - Protein count vs Clan')
plt.show()


# Idea and Initial Implementation
We intend to follow the ideas presented in the ProGen paper relatively closely. That is, our network will be trained on conditioned aminoacid sequences with the only two available conditioning tags being the clan and family IDs. To enable this, our network's input will be as shown on the figure below.

<img src="../Training.png" width="500"/>

So, like ProGen, our input $x=[c;a]$ is a sequence of encoded conditioning tags $c=(c_0,...,c_n)$ (in this case just two), and a starting sequence $a=(a_0, ..., a_n)$ of aminoacids for starting the sequence generation. The sequence is fed one character at a time to the network, which accumulates state before starting generation. We intend to let the network run free either until it generates an end character or until the generated sequence has length 500.


## One-hot encoding
One way to represent a fixed amount of words is through one-hot encoding. We intend to use one-hot encoding for both aminoacids as well as conditioning tags meaning that the input characters to our network become rather large. This may seem inefficient due to the great sparsity in the vectors produced, but we intend to limit this sparsity by decreasing the number of clans and families considered. This will allow us to generate proteins only for a subset of the available clans and families but at a much shorter required training and evaluation time. 

An example of a one-hot encoding is shown below:

| Amionacid    | one-hot encoded vector   |
| ------------- |--------------------------|
| Ala = A        | $= [1, 0, 0, \ldots, 0]$ |
| Asx = B        | $= [0, 1, 0, \ldots, 0]$ |
| Cys = C    | $= [0, 0, 1, \ldots, 0]$ |
| ... | ... |
| pad   | $= [0, 0, 0, \ldots, 1]$ |


In [ ]:
# The following is taken from EXE 5.1 RNN
def one_hot_encode(idx, vocab_size):
    """
    One-hot encodes a single word given its index and the size of the vocabulary.
    
    Args:
     `idx`: the index of the given word
     `vocab_size`: the size of the vocabulary
    
    Returns a 1-D numpy array of length `vocab_size`.
    """
    # Initialize the encoded array
    one_hot = np.zeros(vocab_size)
    
    # Set the appropriate element to one
    one_hot[idx] = 1.0

    return one_hot


def one_hot_encode_sequence(sequence, vocab_size):
    """
    One-hot encodes a sequence of words given a fixed vocabulary size.
    
    Args:
     `sentence`: a list of words to encode
     `vocab_size`: the size of the vocabulary
     
    Returns a 3-D numpy array of shape (num words, vocab size, 1).
    """
    # Encode each word in the sentence
    encoding = np.array([one_hot_encode(word, vocab_size) for word in sequence])

    # Reshape encoding s.t. it has shape (num words, vocab size, 1)
    encoding = encoding.reshape(encoding.shape[0], encoding.shape[1], 1)
    
    return encoding
# End of EXE 5.1 RNN

# Testing the implementation
vocab_size = 30
test_word = one_hot_encode(1, vocab_size)
print(f'Our one-hot encoding of \'1\' has shape {test_word.shape}.')
print(test_word)

test_sentence = one_hot_encode_sequence(holdout_data[1][0], vocab_size)
print(f'Our one-hot encoding of \'{holdout_data[1][0]}\' has shape {test_sentence.shape}.')
print(test_sentence)

## Reducing input size
To reduce training time and problem size, we intend to extract 10 clans and their families from the training set and use those for training the network. Data is stored in sorted order in the training data set meaning that we can simply store elements until we see the 11th clan ID at which we can stop looking through the dataset. This is preferable over having to look through the entire dataset (as we did to gather plot data earlier), as such an operation takes a long time due to the mere size of the dataset.

As part of this operation, we also throw away the input masks that are included in the original dataset and attempt to limit data size by using `int8` instead of the original `int64` format for encoded sequences.

In [ ]:
def get_data(dataset, num):
    """
    Fetches data for `num` clans in the given dataset

    Args:
     `dataset`: an LMDB dataset
     `num`: a number of clans to collect data for

    Returns a DataFrame object with the fetched data
    """
    res = {'Sequence' : [], 'Clan ID' : [], 'Family ID' : []}
    clans = set()
    for i in range(len(dataset)):
        # Fetch this entry
        row = dataset[i]
        seq = row[0].astype('int8')
        clan = row[2]
        family = row[3]

        # Check whether this clan is the (num+1)th
        clans.add(clan)
        if len(clans) > num:
            break

        # The clan was not the (num+1)th; store its data
        res['Sequence'].append(seq)
        res['Clan ID'].append(clan)
        res['Family ID'].append(family)

    return pd.DataFrame.from_dict(res)

## RNN Network for Protein Generation
This section will define the network architecture for the neural network RNN used as the backbone of the ProWave neural network for protein generation. Note that this network serves as a baseline implementation meant to test our ideas, while the final network we desire to implement, is a WaveNet-based model, which differs significantly from the network below.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

clanidsize = 623
familyids = 15000

class MyRecurrentNet(nn.Module):
    def __init__(self):
        super(MyRecurrentNet, self).__init__()
        
        # Recurrent layer
        self.lstm = nn.LSTM(input_size=(vocab_size),
                         hidden_size=1000,
                         num_layers=1,
                         bidirectional=False)
        
        # Output layer
        self.l_out = nn.Linear(in_features=1000,
                            out_features=vocab_size,
                            bias=False)
        
    def forward(self, x):
        # RNN returns output and last hidden state
        
        x, (h, c) = self.lstm(x)
        
        # Flatten output for feed-forward layer
        x = x.view(-1, self.lstm.hidden_size)
        
        # Output layer
        x = self.l_out(x)
        
        return x

net = MyRecurrentNet()
print(net)

### Training loop

It's time for us to train our network. In the section below, you will get to put your deep learning skills to use and create your own training loop. You may want to consult previous exercises if you cannot recall how to define the training loop.

In [ ]:
# Hyper-parameters
num_epochs = 200

# Initialize a new network
net = MyRecurrentNet()

import torch.optim as optim

# Define a loss function and optimizer for this problem
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.004, momentum=0.6) 

# Track loss
training_loss, validation_loss = [], []

# For each epoch
for i in range(num_epochs):
    
    # Track loss
    epoch_training_loss = 0
    epoch_validation_loss = 0
    
    net.eval()
        
    # For each sentence in validation set
    for inputs, targets in validation_set:
        
        # One-hot encode input and target sequence
        inputs_one_hot = one_hot_encode_sequence(inputs, vocab_size)
        targets_idx = [word_to_idx[word] for word in targets]
        
        # Convert input to tensor
        inputs_one_hot = torch.Tensor(inputs_one_hot)
        inputs_one_hot = inputs_one_hot.permute(0, 2, 1)
        
        # Convert target to tensor
        targets_idx = torch.LongTensor(targets_idx)
        
        # Forward pass
        # zero the parameter gradients
        optimizer.zero_grad()
        output= net(inputs_one_hot)
        batch_loss = criterion(output, targets_idx)

        # forward + backward + optimize
        batch_loss.backward()
        optimizer.step() 
        
        # Compute loss
        # YOUR CODE HERE!
        loss = batch_loss
        
        # Update loss
        epoch_validation_loss += loss.detach().numpy()
    
    net.train()
    
    # For each sentence in training set
    for inputs, targets in training_set:
        
        # One-hot encode input and target sequence
        inputs_one_hot = one_hot_encode_sequence(inputs, vocab_size)
        targets_idx = [word_to_idx[word] for word in targets]
        
        # Convert input to tensor
        inputs_one_hot = torch.Tensor(inputs_one_hot)
        inputs_one_hot = inputs_one_hot.permute(0, 2, 1)
        
        # Convert target to tensor
        targets_idx = torch.LongTensor(targets_idx)
        
        # Forward pass
        # zero the parameter gradients
        optimizer.zero_grad()
        output= net(inputs_one_hot)
        batch_loss = criterion(output, targets_idx)

        # forward + backward + optimize
        batch_loss.backward()
        optimizer.step() 
        
        # Compute loss
        # YOUR CODE HERE!
        loss = batch_loss
        
        # Update loss
        epoch_training_loss += loss.detach().numpy()
        
    # Save loss for plot
    training_loss.append(epoch_training_loss/len(training_set))
    validation_loss.append(epoch_validation_loss/len(validation_set))

    # Print loss every 10 epochs
    if i % 10 == 0:
        print(f'Epoch {i}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')

        
# Get first sentence in test set
inputs, targets = test_set[1]

# One-hot encode input and target sequence
inputs_one_hot = one_hot_encode_sequence(inputs, vocab_size)
targets_idx = [word_to_idx[word] for word in targets]

# Convert input to tensor
inputs_one_hot = torch.Tensor(inputs_one_hot)
inputs_one_hot = inputs_one_hot.permute(0, 2, 1)

# Convert target to tensor
targets_idx = torch.LongTensor(targets_idx)

# Forward pass
outputs = net.forward(inputs_one_hot).data.numpy()

print('\nInput sequence:')
print(inputs)

print('\nTarget sequence:')
print(targets)

print('\nPredicted sequence:')
print([idx_to_word[np.argmax(output)] for output in outputs])

# Plot training and validation loss
epoch = np.arange(len(training_loss))
plt.figure()
plt.plot(epoch, training_loss, 'r', label='Training loss',)
plt.plot(epoch, validation_loss, 'b', label='Validation loss')
plt.legend()
plt.xlabel('Epoch'), plt.ylabel('NLL')
plt.show()